In [1]:
import pandas as pd
import numpy as np
import json
import pandas as pd
import re
from rdkit import Chem
from rdkit.Chem import AllChem
from collections import Counter,defaultdict
from ast import literal_eval
import cobra
from cobra import Metabolite,Reaction,Gene
from tqdm import tqdm
import random
import pickle
from multiprocessing import Pool

import sys
# sys.path.append('../Script/')
from common import *
# from check_newinfo_getyeast8U import *

In [2]:
### input
yeast870_path =  '../../Data/model/yeast-GEM.yml'
metabolites_info_to_GEM_path = '../../Results/not_lipid/top50_0.3_re/metabolites_info_to_GEM_top50_0.3.csv'
rxndb_total_info_to_model_path = '../../Results/not_lipid/top50_0.3_re/rxndb_total_info_to_model_top50_0.3.csv'
yeast8_reaction_in_rxndb_json = '../../Results/not_lipid/top50_0.3_re/yeast8_reaction_in_rxndb_top50_0.3.json'

### output
rxndb_total_info_to_model_del_reaction_path = '../../Results/not_lipid/top50_0.3_re/rxndb_total_info_to_model_del_reaction_top50_0.3.csv'
error_reaction_path = '../../Results/not_lipid/top50_0.3_re/error_reaction_random/'
error_reaction_all_path = '../../Results/not_lipid/top50_0.3_re/error_reaction_all.pickle'
yeast8U_del_path = '../../Data/model/yeast8U_del_re.yml'
yeast8U_mod_path = '../../Data/model/yeast8U_mod_re.yml'
biomass_up_reaction_path = '../../Results/not_lipid/top50_0.3_re/biomass_up_reaction_top50_0.3.pickle'

In [3]:
rxndb_total_info_to_model = pd.read_csv(rxndb_total_info_to_model_path)
rxndb_total_info_to_model.shape

(8114, 8)

In [4]:
get_yeast8U(metabolites_info_to_GEM_path,rxndb_total_info_to_model_path,yeast8_reaction_in_rxndb_json,yeast870_path,yeast8U_mod_path)

(4337, 4)
(8114, 8)
(8114, 8)
(6740, 8)


100%|██████████| 6740/6740 [00:25<00:00, 260.17it/s]


In [5]:
yeast8U_mod = cobra.io.load_yaml_model(yeast8U_mod_path)
yeast8U_mod.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
s_0420,r_1654,0.9952,0,0.00%
s_0565,r_1714,1,6,100.00%
s_0796,r_1832,0.07959,0,0.00%
s_0925,r_1861,4.371E-06,0,0.00%
s_1324,r_2005,0.06967,0,0.00%
s_1374,r_2020,0.0005053,0,0.00%
s_1438,r_2049,0.0005526,0,0.00%
s_1468,r_2060,0.002784,0,0.00%
s_4200,r_4593,0.0001796,0,0.00%
s_4201,r_4594,9.173E-05,0,0.00%


In [6]:
yeast8 = cobra.io.load_yaml_model(yeast870_path)


In [7]:
yeast8U_mod_path

'../../Data/model/yeast8U_mod_re.yml'

In [8]:
yeast8U_mod

Name,None
Memory address,7f1f5aab9bb0
Number of metabolites,3261
Number of reactions,10871
Number of genes,2071
Number of groups,0
Objective expression,1.0*r_2111 - 1.0*r_2111_reverse_58b69
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


# find reactions that affect biomass

In [9]:
reaction_id = []
for i in yeast8U_mod.reactions:
    if 'rxn' in i.id:
        i.bounds = (0,0)
        reaction_id.append(i.id)

biomass_up_reaction = []
for i in tqdm(reaction_id,total=len(reaction_id)):
    with yeast8U_mod:
        yeast8U_mod.reactions.get_by_id(i).bounds = (0,1000)
        yeast8U_mod.solver = 'gurobi'
        yeast8U_mod.optimize()
        if yeast8U_mod.reactions.get_by_id('r_2111').flux > 0.09:
            biomass_up_reaction.append(i)
            print(i,'error_reaction')
        else: 
            pass
        
biomass_up_reaction

  8%|▊         | 514/6740 [01:33<19:17,  5.38it/s]

rxn27431 error_reaction


  8%|▊         | 522/6740 [01:34<19:10,  5.40it/s]

rxn27511 error_reaction


  8%|▊         | 525/6740 [01:35<23:00,  4.50it/s]

...

rxn196 error_reaction


 97%|█████████▋| 6559/6740 [21:33<00:34,  5.17it/s]

rxn848 error_reaction


100%|██████████| 6740/6740 [22:07<00:00,  5.08it/s]


['rxn27431',
 'rxn27511',
 'rxn27517',
 'rxn27519',
 'rxn27523',
 'rxn27529',
 'rxn31268',
 'rxn31456',
 'rxn31457',
 'rxn41050',
 'rxn23661',
 'rxn46501',
 'rxn45306',
 'rxn48191',
 'rxn48229',
 'rxn48255',
 'rxn43518',
 'rxn43519',
 'rxn15723',
 'rxn25873',
 'rxn26378',
 'rxn63692',
 'rxn63758',
 'rxn63759',
 'rxn65115',
 'rxn65125',
 'rxn65140',
 'rxn18033',
 'rxn9611',
 'rxn9615',
 'rxn79846',
 'rxn79858',
 'rxn79870',
 'rxn79882',
 'rxn79894',
 'rxn79906',
 'rxn79918',
 'rxn79930',
 'rxn7653',
 'rxn7808',
 'rxn7810',
 'rxn5976',
 'rxn2354',
 'rxn2355',
 'rxn10',
 'rxn196',
 'rxn848']

In [10]:
# Save the list as a pickle file
with open(biomass_up_reaction_path, 'wb') as f:
    pickle.dump(biomass_up_reaction, f)

In [11]:

# # Load the pickle file
with open(biomass_up_reaction_path, 'rb') as f:
    biomass_up_reaction = pickle.load(f)

# View the loaded data
print(biomass_up_reaction)

['rxn27431', 'rxn27511', 'rxn27517', 'rxn27519', 'rxn27523', 'rxn27529', 'rxn31268', 'rxn31456', 'rxn31457', 'rxn41050', 'rxn23661', 'rxn46501', 'rxn45306', 'rxn48191', 'rxn48229', 'rxn48255', 'rxn43518', 'rxn43519', 'rxn15723', 'rxn25873', 'rxn26378', 'rxn63692', 'rxn63758', 'rxn63759', 'rxn65115', 'rxn65125', 'rxn65140', 'rxn18033', 'rxn9611', 'rxn9615', 'rxn79846', 'rxn79858', 'rxn79870', 'rxn79882', 'rxn79894', 'rxn79906', 'rxn79918', 'rxn79930', 'rxn7653', 'rxn7808', 'rxn7810', 'rxn5976', 'rxn2354', 'rxn2355', 'rxn10', 'rxn196', 'rxn848']


In [12]:
atp_reaction = []

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_0434' in tmp and tmp['s_0434']>0:
            atp_reaction.append(reaction.id)
            
print(len(atp_reaction))
print(atp_reaction)

108
['rxn24278', 'rxn24279', 'rxn30704', 'rxn30705', 'rxn30735', 'rxn30736', 'rxn28900', 'rxn29094', 'rxn29095', 'rxn29096', 'rxn29098', 'rxn29099', 'rxn29100', 'rxn35831', 'rxn39408', 'rxn39587', 'rxn37381', 'rxn41902', 'rxn41904', 'rxn41908', 'rxn41910', 'rxn41912', 'rxn22568', 'rxn38833', 'rxn43079', 'rxn44714', 'rxn44716', 'rxn44724', 'rxn34244', 'rxn34245', 'rxn34247', 'rxn34248', 'rxn34249', 'rxn50904', 'rxn51307', 'rxn44671', 'rxn44672', 'rxn44674', 'rxn44675', 'rxn44676', 'rxn44683', 'rxn44684', 'rxn44686', 'rxn44688', 'rxn44700', 'rxn44702', 'rxn44704', 'rxn16782', 'rxn16784', 'rxn25313', 'rxn25315', 'rxn25398', 'rxn52782', 'rxn64111', 'rxn64116', 'rxn64119', 'rxn66057', 'rxn72230', 'rxn17459', 'rxn17460', 'rxn17461', 'rxn17747', 'rxn11478', 'rxn11479', 'rxn11484', 'rxn11486', 'rxn11488', 'rxn11489', 'rxn11492', 'rxn80506', 'rxn80758', 'rxn80787', 'rxn80937', 'rxn81045', 'rxn81232', 'rxn81261', 'rxn81420', 'rxn4525', 'rxn4527', 'rxn4528', 'rxn6156', 'rxn6157', 'rxn6218', 'rxn7

In [13]:
atp_sim_name = ['UDP','UMP','UTP',
                'dUDP','dUMP','dUTP',
                'CDP','CMP','CTP',
                'dCDP','dCMP','dCTP',
                'TDP','TMP','TTP',
                'dTDP','dTMP','dTTP',
                'GDP','GMP','GTP',
                'dGDP','dGMP','dGTP',
                'IDP','IMP','ITP',
                'dIDP','dIMP','dITP',
                ]
atp_sim_list = []
for met in atp_sim_name:
    for i in yeast8.metabolites:
        if i.compartment == 'c':
            if i.name == met:
                print(i.id,i.name,met)
                atp_sim_list.append(i.id)

atp_sim_reaction = []

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if any(x in tmp for x in atp_sim_list):
            atp_sim_reaction.append(reaction.id)
print(len(atp_sim_reaction))
print(atp_sim_reaction)

s_1538 UDP UDP
s_1545 UMP UMP
s_1559 UTP UTP
s_0652 dUDP dUDP
s_0654 dUMP dUMP
s_0656 dUTP dUTP
s_0467 CDP CDP
s_0526 CMP CMP
s_0539 CTP CTP
s_0587 dCDP dCDP
s_0589 dCMP dCMP
s_0590 dCTP dCTP
s_1475 TDP TDP
s_1497 TMP TMP
s_0647 dTDP dTDP
s_0649 dTMP dTMP
s_0650 dTTP dTTP
s_0739 GDP GDP
s_0782 GMP GMP
s_0785 GTP GTP
s_0613 dGDP dGDP
s_0615 dGMP dGMP
s_0617 dGTP dGTP
s_0846 IDP IDP
s_0849 IMP IMP
s_0950 ITP ITP
s_0618 dIDP dIDP
s_0639 dITP dITP
1962
['rxn21219', 'rxn22454', 'rxn22455', 'rxn19749', 'rxn19757', 'rxn20205', 'rxn20206', 'rxn21095', 'rxn21096', 'rxn21097', 'rxn21098', 'rxn21101', 'rxn21102', 'rxn21103', 'rxn24519', 'rxn24520', 'rxn24521', 'rxn24525', 'rxn24527', 'rxn24529', 'rxn24530', 'rxn24531', 'rxn24534', 'rxn24535', 'rxn24536', 'rxn24537', 'rxn24541', 'rxn24543', 'rxn24545', 'rxn24546', 'rxn24547', 'rxn24550', 'rxn24631', 'rxn24632', 'rxn24633', 'rxn24634', 'rxn24635', 'rxn24636', 'rxn24637', 'rxn24638', 'rxn24639', 'rxn24640', 'rxn24641', 'rxn24642', 'rxn24643', 'rxn24

In [14]:
co2_reaction = []

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_0456' in tmp and tmp['s_0456']<0:
            co2_reaction.append(reaction.id)

print(len(co2_reaction))
print(co2_reaction)

395
['rxn20624', 'rxn20625', 'rxn20631', 'rxn20632', 'rxn20633', 'rxn24199', 'rxn24822', 'rxn24823', 'rxn24826', 'rxn24827', 'rxn24830', 'rxn24831', 'rxn24834', 'rxn24836', 'rxn24838', 'rxn24839', 'rxn24840', 'rxn24842', 'rxn24843', 'rxn24844', 'rxn24846', 'rxn24847', 'rxn24848', 'rxn24850', 'rxn24851', 'rxn24852', 'rxn24854', 'rxn24855', 'rxn24856', 'rxn24858', 'rxn24859', 'rxn24947', 'rxn24948', 'rxn30507', 'rxn26905', 'rxn26906', 'rxn26908', 'rxn27511', 'rxn27513', 'rxn27515', 'rxn27517', 'rxn27519', 'rxn27521', 'rxn27523', 'rxn27525', 'rxn27527', 'rxn27529', 'rxn27531', 'rxn27533', 'rxn27535', 'rxn27537', 'rxn27539', 'rxn27541', 'rxn27543', 'rxn27545', 'rxn27547', 'rxn27549', 'rxn18763', 'rxn22599', 'rxn22600', 'rxn38434', 'rxn38435', 'rxn38849', 'rxn39077', 'rxn39078', 'rxn42736', 'rxn42737', 'rxn42738', 'rxn46499', 'rxn46500', 'rxn46501', 'rxn50712', 'rxn50713', 'rxn50714', 'rxn50715', 'rxn50716', 'rxn43614', 'rxn15723', 'rxn15724', 'rxn15725', 'rxn16584', 'rxn25429', 'rxn52524',

In [15]:
o2_reaction = []

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_1275' in tmp and tmp['s_1275']>0:
            o2_reaction.append(reaction.id)

print(len(o2_reaction))
print(o2_reaction)

568
['rxn21142', 'rxn21581', 'rxn21607', 'rxn21633', 'rxn21659', 'rxn21685', 'rxn21711', 'rxn21724', 'rxn19958', 'rxn19959', 'rxn19960', 'rxn19961', 'rxn20861', 'rxn20863', 'rxn24822', 'rxn24823', 'rxn24826', 'rxn24827', 'rxn24830', 'rxn24831', 'rxn24834', 'rxn24836', 'rxn24838', 'rxn24839', 'rxn24840', 'rxn24842', 'rxn24843', 'rxn24844', 'rxn24846', 'rxn24847', 'rxn24848', 'rxn24850', 'rxn24851', 'rxn24852', 'rxn24854', 'rxn24855', 'rxn24856', 'rxn24858', 'rxn24859', 'rxn29553', 'rxn29778', 'rxn30367', 'rxn30616', 'rxn30617', 'rxn30627', 'rxn30629', 'rxn27015', 'rxn27134', 'rxn27431', 'rxn27511', 'rxn27513', 'rxn27515', 'rxn27517', 'rxn27519', 'rxn27521', 'rxn27523', 'rxn27525', 'rxn27527', 'rxn27529', 'rxn27531', 'rxn27533', 'rxn27535', 'rxn27537', 'rxn27539', 'rxn27541', 'rxn27543', 'rxn27545', 'rxn27547', 'rxn27549', 'rxn27946', 'rxn28226', 'rxn28227', 'rxn28228', 'rxn28229', 'rxn28230', 'rxn28231', 'rxn28232', 'rxn28234', 'rxn29118', 'rxn29119', 'rxn29120', 'rxn29315', 'rxn29316',

In [16]:
nadph_reaction = []
# nadph nadh fadh2

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_1212' in tmp and tmp['s_1212']>0:  #NADPH
            nadph_reaction.append(reaction.id)

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_1203' in tmp and tmp['s_1203']>0:   #NADH
            nadph_reaction.append(reaction.id)

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_0689' in tmp and tmp['s_0689']>0:  #FADH2
            nadph_reaction.append(reaction.id)

print(len(nadph_reaction))
print(nadph_reaction)


587
['rxn21299', 'rxn22397', 'rxn22398', 'rxn22399', 'rxn19912', 'rxn19913', 'rxn19914', 'rxn20861', 'rxn20863', 'rxn21104', 'rxn21105', 'rxn24511', 'rxn24512', 'rxn24513', 'rxn24514', 'rxn24515', 'rxn24516', 'rxn24517', 'rxn24518', 'rxn30425', 'rxn30704', 'rxn30705', 'rxn27015', 'rxn27055', 'rxn27946', 'rxn35812', 'rxn14954', 'rxn15011', 'rxn15012', 'rxn15013', 'rxn15087', 'rxn15352', 'rxn36471', 'rxn37397', 'rxn30905', 'rxn31171', 'rxn31268', 'rxn31269', 'rxn31456', 'rxn31921', 'rxn31924', 'rxn31925', 'rxn18407', 'rxn18848', 'rxn18850', 'rxn18852', 'rxn18854', 'rxn18856', 'rxn18858', 'rxn18860', 'rxn18862', 'rxn18864', 'rxn18866', 'rxn18868', 'rxn18870', 'rxn18872', 'rxn18874', 'rxn18876', 'rxn18877', 'rxn18880', 'rxn18882', 'rxn18884', 'rxn18885', 'rxn18888', 'rxn18890', 'rxn18891', 'rxn18902', 'rxn18903', 'rxn18906', 'rxn18908', 'rxn18924', 'rxn18926', 'rxn19515', 'rxn22546', 'rxn22552', 'rxn22777', 'rxn38836', 'rxn38897', 'rxn38898', 'rxn38899', 'rxn38944', 'rxn46532', 'rxn46535',

In [17]:
# ['s_1360','s_0075','s_4092','s_1464'] 

yeast8_atp = []

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_1360' in tmp and tmp['s_1360']>0:   #phosphoenolpyruvate
            yeast8_atp.append(reaction.id)

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_0075' in tmp and tmp['s_0075']>0:   #1,3-bisphospho-D-glycerate
            yeast8_atp.append(reaction.id)

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_4092' in tmp and tmp['s_4092']>0:  #N(omega)-phospho-L-arginine
            yeast8_atp.append(reaction.id)

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_1464' in tmp and tmp['s_1464']>0:  #succinyl-CoA
            yeast8_atp.append(reaction.id)


print(len(yeast8_atp))
print(yeast8_atp)

64
['rxn21095', 'rxn21096', 'rxn21097', 'rxn21098', 'rxn21101', 'rxn21102', 'rxn21103', 'rxn34817', 'rxn34818', 'rxn26376', 'rxn26377', 'rxn26378', 'rxn26380', 'rxn26382', 'rxn26383', 'rxn68262', 'rxn68263', 'rxn68266', 'rxn68267', 'rxn69570', 'rxn69571', 'rxn69574', 'rxn69575', 'rxn69872', 'rxn69875', 'rxn82748', 'rxn82749', 'rxn82754', 'rxn82766', 'rxn82768', 'rxn5750', 'rxn27295', 'rxn27313', 'rxn27369', 'rxn27387', 'rxn28707', 'rxn28725', 'rxn37595', 'rxn25948', 'rxn25966', 'rxn25984', 'rxn26002', 'rxn26026', 'rxn26044', 'rxn26064', 'rxn26082', 'rxn26100', 'rxn26118', 'rxn12810', 'rxn12811', 'rxn12835', 'rxn12883', 'rxn12914', 'rxn12915', 'rxn12666', 'rxn12667', 'rxn12683', 'rxn12699', 'rxn12715', 'rxn12746', 'rxn12747', 'rxn82979', 'rxn12606', 'rxn81965']


In [18]:
else_reaction = []

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_0764' in tmp and tmp['s_0764']>0:     #glycerol
            else_reaction.append(reaction.id)

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_0362' in tmp and tmp['s_0362']<0:   #acetate
            else_reaction.append(reaction.id)

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_0140' in tmp and tmp['s_0140']>0:  #2,3-bisphospho-D-glyceric acid
            else_reaction.append(reaction.id)

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_0979' in tmp and tmp['s_0979']<0:  #L-citrulline
            else_reaction.append(reaction.id)

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_0373' in tmp and tmp['s_0373']>0:  #acetyl-CoA
            else_reaction.append(reaction.id)

print(len(else_reaction))
print(else_reaction)

244
['rxn14363', 'rxn37249', 'rxn37594', 'rxn19684', 'rxn19688', 'rxn12804', 'rxn12805', 'rxn12830', 'rxn12878', 'rxn12908', 'rxn12909', 'rxn12978', 'rxn13080', 'rxn14062', 'rxn14080', 'rxn56569', 'rxn56573', 'rxn74656', 'rxn74672', 'rxn74688', 'rxn74704', 'rxn74720', 'rxn74736', 'rxn74752', 'rxn74768', 'rxn74784', 'rxn74800', 'rxn12660', 'rxn12661', 'rxn12680', 'rxn12696', 'rxn12712', 'rxn12740', 'rxn12741', 'rxn7808', 'rxn1661', 'rxn129', 'rxn139', 'rxn20696', 'rxn20697', 'rxn24263', 'rxn24844', 'rxn24846', 'rxn24847', 'rxn25241', 'rxn30677', 'rxn27048', 'rxn27523', 'rxn27543', 'rxn35646', 'rxn35801', 'rxn35953', 'rxn14647', 'rxn39625', 'rxn37407', 'rxn37495', 'rxn30960', 'rxn31155', 'rxn31892', 'rxn31901', 'rxn31913', 'rxn19126', 'rxn19127', 'rxn22992', 'rxn38301', 'rxn38839', 'rxn38914', 'rxn38915', 'rxn38916', 'rxn38918', 'rxn38920', 'rxn38923', 'rxn45305', 'rxn50643', 'rxn16460', 'rxn50076', 'rxn55458', 'rxn55459', 'rxn25704', 'rxn25876', 'rxn26178', 'rxn26185', 'rxn26314', 'rxn1

# Calculate growth after preliminary screening

In [19]:
reaction_id = []
for i in yeast8U_mod.reactions:
    if 'rxn' in i.id:
        i.bounds = (0,0)
        reaction_id.append(i.id)

reaction_id = [x for x in reaction_id if x not in biomass_up_reaction]
reaction_id = [x for x in reaction_id if x not in atp_reaction]
reaction_id = [x for x in reaction_id if x not in atp_sim_reaction]
reaction_id = [x for x in reaction_id if x not in co2_reaction]
reaction_id = [x for x in reaction_id if x not in o2_reaction]
reaction_id = [x for x in reaction_id if x not in nadph_reaction]
reaction_id = [x for x in reaction_id if x not in yeast8_atp]
reaction_id = [x for x in reaction_id if x not in else_reaction]


# for seed_num in range(20,40):
#     random.seed(seed_num)
#     random.shuffle(reaction_id)
#     with yeast8U_mod:
#         error_reaction_4 = []
#         for i in tqdm(reaction_id):
#             reaction = yeast8U_mod.reactions.get_by_id(i)
#             if 'rxn' in reaction.id:
#                 reaction.bounds = (0,1000)
#                 yeast8U_mod.solver = 'gurobi'
#                 yeast8U_mod.optimize()
#                 if yeast8U_mod.reactions.get_by_id('r_2111').flux > 0.1:
#                     reaction.bounds = (0,0)
#                     error_reaction_4.append(reaction.id)
#         print(seed_num,len(error_reaction_4))
#     # Store the list in a binary file
#     filename = error_reaction_path + 'error_reaction_random'+str(seed_num)+'.pkl'
#     with open(filename, 'wb') as file:
#         pickle.dump(error_reaction_4, file)

In [21]:


def process_seed(args):
    seed_num, reaction_id, yeast8U_mod, error_reaction_path = args
    random.seed(seed_num)
    random.shuffle(reaction_id)
    with yeast8U_mod:
        error_reaction_4 = []
        for i in tqdm(reaction_id):
            reaction = yeast8U_mod.reactions.get_by_id(i)
            if 'rxn' in reaction.id:
                reaction.bounds = (0, 1000)
                yeast8U_mod.solver = 'gurobi'
                yeast8U_mod.optimize()
                if yeast8U_mod.reactions.get_by_id('r_2111').flux > 0.09:
                    reaction.bounds = (0, 0)
                    error_reaction_4.append(reaction.id)
        print(seed_num, len(error_reaction_4))
    # Store the list in a binary file
    filename = error_reaction_path + 'error_reaction_random' + str(seed_num) + '.pkl'
    if not os.path.exists(error_reaction_path):
        os.makedirs(error_reaction_path)
    with open(filename, 'wb') as file:
        pickle.dump(error_reaction_4, file)



if __name__ == '__main__':
    with Pool(20) as pool:
        args = [(seed_num, reaction_id, yeast8U_mod, error_reaction_path) for seed_num in range(20, 40)]
        results = list(tqdm(pool.imap(process_seed, args), total=len(args)))

  0%|          | 0/20 [00:00<?, ?it/s]

Read LP format model from file /tmp/tmpcslg98q6.lp
Reading time = 0.04 seconds
: 3261 rows, 21742 columns, 99128 nonzeros
Read LP format model from file /tmp/tmpmupu50p5.lp
Reading time = 0.04 seconds
: 3261 rows, 21742 columns, 99128 nonzeros
Read LP format model from file /tmp/tmpgof_l_cl.lp
Reading time = 0.06 seconds
: 3261 rows, 21742 columns, 99128 nonzeros
Read LP format model from file /tmp/tmpidsvjxvr.lp
Reading time = 0.04 seconds
: 3261 rows, 21742 columns, 99128 nonzeros
Read LP format model from file /tmp/tmpa4e507wn.lp
Reading time = 0.04 seconds
: 3261 rows, 21742 columns, 99128 nonzeros
Read LP format model from file /tmp/tmpudyjqixg.lp
Reading time = 0.04 seconds
: 3261 rows, 21742 columns, 99128 nonzeros
Read LP format model from file /tmp/tmpvhszsni7.lp
Reading time = 0.04 seconds
: 3261 rows, 21742 columns, 99128 nonzeros
Read LP format model from file /tmp/tmpvm396v19.lp
Reading time = 0.04 seconds
: 3261 rows, 21742 columns, 99128 nonzeros
Read LP format model fro

  0%|          | 2/3444 [00:00<17:10,  3.34it/s]

Read LP format model from file /tmp/tmpu40r24fc.lp
Reading time = 0.04 seconds
: 3261 rows, 21742 columns, 99128 nonzeros


  0%|          | 14/3444 [00:01<06:27,  8.85it/s]

...

100%|██████████| 3444/3444 [07:51<00:00,  7.31it/s]


28 148


100%|██████████| 3444/3444 [06:40<00:00,  8.60it/s]


37 84


100%|██████████| 20/20 [22:30<00:00, 67.50s/it]   


# Select and delete the pkl file with the fewest reactions

In [22]:
# error_reaction_path = "../../Results/not_lipid/top50_0.3/error_reaction_random/"
files = os.listdir(error_reaction_path)

# Find the file with the minimum error_reaction
min_error_reaction = 1000
min_error_reaction_file = None

for file in files:
    file_path = os.path.join(error_reaction_path, file)
    with open(file_path, 'rb') as f:
        error_reaction_data = pickle.load(f)
        if len(error_reaction_data) < min_error_reaction:
            min_error_reaction = len(error_reaction_data)
            min_error_reaction_file = file  # Store the file name

print(min_error_reaction_file)


file_path = os.path.join(error_reaction_path, min_error_reaction_file)
with open(file_path, 'rb') as f:
    error_reaction_data = pickle.load(f)
print(f"Loaded data length: {len(error_reaction_data)}")


error_reaction_random34.pkl
Loaded data length: 48


In [23]:
error_reaction_data

['rxn11226',
 'rxn6183',
 'rxn44080',
 'rxn1488',
 'rxn13374',
 'rxn12112',
 'rxn65612',
 'rxn2712',
 'rxn1670',
 'rxn2961',
 'rxn11997',
 'rxn24304',
 'rxn38832',
 'rxn3535',
 'rxn70098',
 'rxn21568',
 'rxn1489',
 'rxn45922',
 'rxn6191',
 'rxn11556',
 'rxn566',
 'rxn11166',
 'rxn52860',
 'rxn40192',
 'rxn25539',
 'rxn27478',
 'rxn2231',
 'rxn11186',
 'rxn7809',
 'rxn45913',
 'rxn45915',
 'rxn25475',
 'rxn14067',
 'rxn5927',
 'rxn22936',
 'rxn68634',
 'rxn167',
 'rxn11106',
 'rxn26933',
 'rxn83810',
 'rxn11126',
 'rxn5792',
 'rxn3147',
 'rxn70096',
 'rxn846',
 'rxn34014',
 'rxn31540',
 'rxn2950']

In [24]:
del_reaction_lst = biomass_up_reaction + atp_reaction + atp_sim_reaction + co2_reaction + o2_reaction + nadph_reaction + yeast8_atp + else_reaction + error_reaction_data
print(len(del_reaction_lst))
del_reaction_lst = list(set(del_reaction_lst))
print(len(del_reaction_lst))

4023
3344


In [25]:
del_reaction_lst

['rxn11140',
 'rxn2496',
 'rxn24645',
 'rxn2355',
 'rxn63760',
 'rxn17289',
 'rxn2854',
 'rxn15558',
 'rxn8011',
 'rxn4040',
 'rxn4708',
 'rxn72959',
 'rxn78094',
 'rxn4116',
 'rxn68186',
 'rxn81414',
 'rxn1258',
 'rxn6773',
 'rxn30425',
 'rxn13493',
 'rxn62372',
 'rxn1701',
 'rxn11186',
 'rxn11155',
 'rxn5844',
 'rxn20861',
 'rxn64119',
 'rxn24947',
 'rxn73007',
 'rxn52240',
 'rxn33555',
 'rxn24786',
 'rxn26104',
 'rxn2422',
 'rxn27298',
 'rxn80506',
 'rxn27379',
 'rxn3655',
 'rxn2367',
 'rxn2536',
 'rxn13464',
 'rxn12130',
 'rxn29316',
 'rxn1473',
 'rxn4713',
 'rxn2534',
 'rxn21105',
 'rxn79906',
 'rxn74686',
 'rxn29044',
 'rxn4931',
 'rxn27478',
 'rxn633',
 'rxn38915',
 'rxn15395',
 'rxn24511',
 'rxn26100',
 'rxn11193',
 'rxn12609',
 'rxn6763',
 'rxn504',
 'rxn17408',
 'rxn5583',
 'rxn24657',
 'rxn2077',
 'rxn83810',
 'rxn531',
 'rxn51667',
 'rxn48214',
 'rxn17180',
 'rxn80567',
 'rxn15529',
 'rxn54621',
 'rxn24537',
 'rxn522',
 'rxn2421',
 'rxn11224',
 'rxn54599',
 'rxn52316',
 'rx

In [26]:
# filename = '../Data/analysis/error_reaction_all.pkl'
with open(error_reaction_all_path, 'wb') as file:
    pickle.dump(del_reaction_lst, file)

In [27]:
# with open(error_reaction_all_path, 'rb') as file:
#     del_reaction_lst_ = pickle.load(file)
# del_reaction_lst_

In [28]:
rxndb_total_info_to_model = pd.read_csv(rxndb_total_info_to_model_path)
print(rxndb_total_info_to_model.shape)
rxndb_total_info_to_model = rxndb_total_info_to_model[~rxndb_total_info_to_model['NO'].isin(del_reaction_lst)]
print(rxndb_total_info_to_model.shape)
rxndb_total_info_to_model.to_csv(rxndb_total_info_to_model_del_reaction_path, index=False)

(8114, 8)
(4770, 8)


In [29]:
get_yeast8U(metabolites_info_to_GEM_path,rxndb_total_info_to_model_del_reaction_path,yeast8_reaction_in_rxndb_json,yeast870_path,yeast8U_del_path)


(4337, 4)
(4770, 8)
(4770, 8)
(3396, 8)


100%|██████████| 3396/3396 [03:01<00:00, 18.66it/s]


In [30]:
yeast8U_del = cobra.io.load_yaml_model(yeast8U_del_path)
yeast8U_del.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
s_0420,r_1654,0.6435,0,0.00%
s_0565,r_1714,1,6,100.00%
s_0796,r_1832,0.04552,0,0.00%
s_0925,r_1861,2.826E-06,0,0.00%
s_1277,r_1992,2.376,0,0.00%
s_1324,r_2005,0.04505,0,0.00%
s_1374,r_2020,0.0003267,0,0.00%
s_1438,r_2049,0.0003573,0,0.00%
s_1468,r_2060,0.007741,0,0.00%
s_4200,r_4593,0.0001161,0,0.00%


In [ ]:
# for reaction_id in tqdm(del_reaction_lst):
#     reaction = yeast8U_mod.reactions.get_by_id(reaction_id)
#     yeast8U_mod.remove_reactions([reaction])

100%|██████████| 3531/3531 [00:58<00:00, 60.33it/s]


In [ ]:
# yeast8U_del_path = '../Data/model/yeast8U_Final_mod_del.yml'

# cobra.io.save_yaml_model(yeast8U_mod,yeast8U_del_path)